In [39]:
!nvidia-smi

Wed May  5 23:31:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    34W / 250W |  10319MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
mv: cannot stat 'kaggle.json': No such file or directory


In [18]:
!pip install kaggle -q
!kaggle datasets download -d aladdinpersson/flickr8kimagescaptions
!unzip -q flickr8kimagescaptions.zip

 99% 1.02G/1.04G [00:13<00:00, 75.5MB/s]
100% 1.04G/1.04G [00:13<00:00, 80.5MB/s]


In [19]:
# get the code for kaggle
!git clone https://github.com/moaaztaha/Image-Captioning
py_files_path = 'Image-Captioning/'
import sys
sys.path.append(py_files_path)

fatal: destination path 'Image-Captioning' already exists and is not an empty directory.


In [20]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [21]:
import time 
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention
from dataset import *
from utils import *
from train import *
from nltk.translate.bleu_score import corpus_bleu

In [51]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 7  # number of epochs to train for (if early stopping is not triggered)
batch_size = 64
workers = 2
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = True  # fine-tune encoder?
checkpoint = None  # path to checkpoint, None if none

In [58]:
DATA_NAME = 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_finetune'

# local
# DATA_JSON_PATH = 'data.json'
# IMGS_PATH = 'flickr/Images/'
# kaggle paths
# DATA_JSON_PATH = '/kaggle/working/Image-Captioning/data.json'
# IMGS_PATH = '../input/flickr8kimagescaptions/flickr8k/images/'
#colab
DATA_JSON_PATH = 'Image-Captioning/data.json'
IMGS_PATH = 'flickr8k/images/'

In [59]:
# load vocab
vocab = build_vocab(DATA_JSON_PATH)

100%|██████████| 40000/40000 [00:00<00:00, 262130.07it/s]


In [60]:
len(vocab)

5089

In [61]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout
}

In [62]:
t_params

{'batch_size': 64,
 'data_name': 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_finetune',
 'decoder_lr': 0.0004,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 0.0001,
 'epochs': 7,
 'fine_tune_encoder': True,
 'imgs_path': 'flickr8k/images/',
 'vocab': <dataset.Vocabulary at 0x7fcd521bd450>,
 'workers': 2}

In [63]:
fit(t_params=t_params, m_params=m_params)

Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [0][0/469]	Batch Time 1.709 (1.709)	Data Load Time 0.980 (0.980)	Loss 9.4528 (9.4528)	Top-5 Accuracy 0.130 (0.130)
Epoch: [0][100/469]	Batch Time 0.733 (0.713)	Data Load Time 0.007 (0.013)	Loss 5.1578 (5.7236)	Top-5 Accuracy 49.387 (42.108)
Epoch: [0][200/469]	Batch Time 0.707 (0.709)	Data Load Time 0.000 (0.007)	Loss 4.7331 (5.3617)	Top-5 Accuracy 55.499 (47.126)
Epoch: [0][300/469]	Batch Time 0.695 (0.706)	Data Load Time 0.000 (0.006)	Loss 4.6288 (5.1572)	Top-5 Accuracy 56.690 (49.977)
Epoch: [0][400/469]	Batch Time 0.690 (0.704)	Data Load Time 0.005 (0.005)	Loss 4.5205 (5.0185)	Top-5 Accuracy 59.290 (51.909)
-------------------- Validation -----------

In [72]:
!cp /content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_finetune.pth.tar /content/drive/MyDrive/weights/

In [66]:
batch_size = 64
fine_tune_encoder = True
checkpoint = '/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_finetune.pth.tar'
epochs = 15

t_params = {
    'data_name': DATA_NAME+'_finetune',
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr*.8,
    'encoder_lr': encoder_lr*.8,
    'fine_tune_encoder': fine_tune_encoder
}
t_params

{'batch_size': 64,
 'data_name': 'flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_finetune_finetune',
 'decoder_lr': 0.00032,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 8e-05,
 'epochs': 15,
 'fine_tune_encoder': True,
 'imgs_path': 'flickr8k/images/',
 'vocab': <dataset.Vocabulary at 0x7fcd521bd450>,
 'workers': 2}

In [67]:
fit(t_params, checkpoint=checkpoint, m_params=m_params)

Loaded Checkpoint!!
Starting Epoch: 5
Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [5][0/469]	Batch Time 1.682 (1.682)	Data Load Time 0.904 (0.904)	Loss 3.1990 (3.1990)	Top-5 Accuracy 77.147 (77.147)
Epoch: [5][100/469]	Batch Time 0.716 (0.714)	Data Load Time 0.000 (0.011)	Loss 3.5080 (3.3834)	Top-5 Accuracy 72.785 (74.127)
Epoch: [5][200/469]	Batch Time 0.717 (0.707)	Data Load Time 0.000 (0.007)	Loss 3.3540 (3.4100)	Top-5 Accuracy 73.752 (73.834)
Epoch: [5][300/469]	Batch Time 0.701 (0.705)	Data Load Time 0.000 (0.005)	Loss 3.2632 (3.4052)	Top-5 Accuracy 76.036 (73.997)
Epoch: [5][400/469]	Batch Time 0.732 (0.704)	Data Load Time 0.011 (0.005)	Loss 3.5175 (3.4101)	Top-5 Accuracy 73.127 (73.942)
---

In [70]:
checkpoint = load_checkpoint('/content/BEST_checkpoint_flickr8k_5_cap_per_img_2_min_word_freq_resnet101_fullvocab_fix_ds_finetune.pth.tar')

Loaded Checkpoint!!
Last Epoch: 4
Best Bleu-4: 0.1548061271231411
